# 3. Potential Outcomes and RCTs

In [ ]:
#pip install numpy pandas scipy statsmodels

   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   -------- ------------------------------- 8.4/38.5 MB 41.1 MB/s eta 0:00:01
   ---------------- ----------------------- 16.3/38.5 MB 38.7 MB/s eta 0:00:01
   ---------------------------- ----------- 27.3/38.5 MB 43.0 MB/s eta 0:00:01
   ---------------------------------------  38.0/38.5 MB 44.9 MB/s eta 0:00:01
   ---------------------------------------  38.3/38.5 MB 44.4 MB/s eta 0:00:01
   ---------------------------------------  38.3/38.5 MB 44.4 MB/s eta 0:00:01
   ---------------------------------------- 38.5/38.5 MB 25.8 MB/s  0:00:01
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------  9.4/9.6 MB 51.5 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 22.4 MB/s  0:00:00

   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
import statsmodels.api as sm
import statsmodels.formula.api as smf

## 3.1 Data Simulation

In [ ]:
# -----------------------------
# 3.1 SIMULACIÓN DE DATOS
# -----------------------------
np.random.seed(123)  # para reproducibilidad
n = 1000

# Covariables
x1 = np.random.normal(0, 1, n)
x2 = np.random.normal(1, 0.5, n)
x3 = np.random.uniform(0, 1, n)
x4 = np.random.normal(5, 2, n)

# Tratamiento D ~ Bernoulli(0.5)
D = np.random.binomial(1, 0.5, n)

# Error
epsilon = np.random.normal(0, 1, n)

# Variable de resultado
Y = 2*D + 0.5*x1 - 0.3*x2 + 0.2*x3 + epsilon

# Crear DataFrame
df = pd.DataFrame({
    "Y": Y, "D": D, "x1": x1, "x2": x2, "x3": x3, "x4": x4
})

print("Primeras filas del dataset simulado:")
print(df.head())

Primeras filas del dataset simulado:
          Y  D        x1  x2        x3        x4
0 -0.839679  0 -1.085631   1  0.854010  3.772356
1  0.989059  1  0.997345   0  0.005682  1.922993
2 -0.116453  0  0.282978   0  0.783901  4.017602
3  0.549798  0 -1.506295   1  0.535970  8.734001
4 -0.310911  0 -0.578600   1  0.437351  7.327609


In [ ]:
# -----------------------------
# 3.1(b) COMPROBACIÓN DE BALANCE
# -----------------------------
print("\n=== Balance de covariables entre Tratamiento y Control ===")
for var in ["x1", "x2", "x3", "x4"]:
    mean_treat = df.loc[df["D"]==1, var].mean()
    mean_control = df.loc[df["D"]==0, var].mean()
    ttest = ttest_ind(df.loc[df["D"]==1, var], df.loc[df["D"]==0, var])
    print(f"{var}: media tratados={mean_treat:.3f}, media control={mean_control:.3f}, p-valor={ttest.pvalue:.3f}")


=== Balance de covariables entre Tratamiento y Control ===
x1: media tratados=-0.020, media control=-0.060, p-valor=0.523
x2: media tratados=0.501, media control=0.499, p-valor=0.950
x3: media tratados=0.504, media control=0.497, p-valor=0.670
x4: media tratados=5.010, media control=5.085, p-valor=0.546


## 3.2 Estimating the Average Treatment Effect

In [6]:
# -----------------------------
# 3.2 ESTIMACIÓN DEL ATE
# -----------------------------
print("\n=== Estimación del ATE con regresión simple (Y ~ D) ===")
model1 = smf.ols("Y ~ D", data=df).fit()
print(model1.summary().tables[1])

print("\n=== Estimación del ATE con controles (Y ~ D + x1 + x2 + x3 + x4) ===")
model2 = smf.ols("Y ~ D + x1 + x2 + x3 + x4", data=df).fit()
print(model2.summary().tables[1])

# Comparación de coeficientes
coef_simple = model1.params["D"]
coef_control = model2.params["D"]
se_simple = model1.bse["D"]
se_control = model2.bse["D"]

print("\n=== Comparación de ATE ===")
print(f"ATE (sin controles): {coef_simple:.3f} (SE={se_simple:.3f})")
print(f"ATE (con controles): {coef_control:.3f} (SE={se_control:.3f})")


=== Estimación del ATE con regresión simple (Y ~ D) ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0599      0.050     -1.201      0.230      -0.158       0.038
D              2.0377      0.070     29.176      0.000       1.901       2.175

=== Estimación del ATE con controles (Y ~ D + x1 + x2 + x3 + x4) ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0960      0.112     -0.861      0.389      -0.315       0.123
D              2.0173      0.063     32.206      0.000       1.894       2.140
x1             0.4354      0.031     13.903      0.000       0.374       0.497
x2            -0.3272      0.063     -5.216      0.000      -0.450      -0.204
x3             0.4404      0.108      4.062      0.000       0.228       0.653
x4  

¿El ATE cambia?
¿Qué pasa con los errores estándar?

El ATE prácticamente no cambia al incluir controles, lo que confirma que la asignación de tratamiento fue aleatoria. Sin embargo, los errores estándar se reducen cuando controlamos covariables, lo que implica una ganancia en eficiencia estadística.

## 3.3 LASSO and Variable Selection

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import VarianceThreshold

import statsmodels.api as sm  # para OLS con errores estándar e IC

def lasso_then_ate(df, y="Y", d="D", xs=("x1","x2","x3","x4"),
                   nfolds=5, random_state=123):
    assert isinstance(df, pd.DataFrame)

    # --- 1) Verificaciones de columnas
    need = [y, d, *xs]
    missing = [c for c in need if c not in df.columns]
    if missing:
        raise ValueError(f"Faltan columnas: {missing}")

    # --- 2) Subset y limpieza básica (filas completas sólo en variables usadas)
    df0 = df[need].copy()
    dfc = df0.dropna(axis=0).copy()
    if dfc.empty:
        raise ValueError("No hay filas completas tras eliminar NA.")

    # --- 3) Matriz X (sin D) e y
    X = dfc.loc[:, xs].astype(float).values
    yv = dfc.loc[:, y].astype(float).values
    Dv = dfc.loc[:, d].values  # entra en el OLS posterior

    # --- 4) Remover varianza ~0 (equivalente al filtro de varianza cero)
    vt = VarianceThreshold(threshold=0.0)
    X_nz = vt.fit_transform(X)
    kept_mask = vt.get_support()
    kept_features = [x for x, keep in zip(xs, kept_mask) if keep]
    if X_nz.shape[1] == 0:
        raise ValueError("No quedan predictores después de remover varianza cero.")

    # --- 5) LASSO con CV (imputación + escalado por si acaso)
    pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("lasso", LassoCV(cv=nfolds, n_alphas=200, random_state=random_state, max_iter=10000))
    ])
    pipe.fit(X_nz, yv)
    lasso = pipe.named_steps["lasso"]

    alpha_min = float(lasso.alpha_)  # λ_min
    coefs = lasso.coef_
    # Ojo: coefs corresponden a kept_features (tras VarianceThreshold)
    selected_vars = [f for f, c in zip(kept_features, coefs) if abs(c) > 1e-10]

    # --- 6) Re-estimar ATE con OLS: Y ~ D + X_selected
    # Construimos el DataFrame para statsmodels
    df_ols = pd.DataFrame({"Y": yv, "D": Dv})
    for v in selected_vars:
        df_ols[v] = dfc[v].values  # mismas filas completas

    # Matriz de diseño con intercepto
    X_ols = sm.add_constant(df_ols[["D"] + selected_vars])
    ols_model = sm.OLS(df_ols["Y"].values, X_ols).fit()

    # Extraer métricas para D
    ate = ols_model.params["D"]
    se = ols_model.bse["D"]
    pval = ols_model.pvalues["D"]
    ci_low, ci_high = ols_model.conf_int().loc["D"].tolist()

    # Devolver resultados
    return {
        "alpha_min": alpha_min,
        "selected_vars": selected_vars,
        "ols_summary": ols_model.summary(),
        "ate": float(ate),
        "se": float(se),
        "pval": float(pval),
        "ci95": (float(ci_low), float(ci_high)),
    }

# ================== EJECUCIÓN ==================
# Supone que ya tienes un DataFrame df con columnas: Y, D, x1, x2, x3, x4
res = lasso_then_ate(df=df, y="Y", d="D", xs=("x1","x2","x3","x4"))

print(f"alpha (lambda) min: {res['alpha_min']:.6g}")
print("Selected at lambda.min:", res["selected_vars"] if res["selected_vars"] else "(none)")
print(f"ATE (coef D) = {res['ate']:.3f}, SE = {res['se']:.3f}, p = {res['pval']:.4f}, "
      f"95% CI = [{res['ci95'][0]:.3f}, {res['ci95'][1]:.3f}]")

# Si quieres ver toda la tabla OLS:
# print(res["ols_summary"])

# Comment

#In this case, with the specific variables we have used, post-LASSO matches OLS with controls—with only 4 predictors and clear signal, LASSO doesn’t need to drop variables at λ_min.
#Why use Lasso?
#in higher-dimensional settings (many X’s, interactions, polynomials), LASSO keeps prognostic covariates and drops noise.
#Better generalization & stability: mitigates overfitting and collinearity; can yield smaller variance for the ATE after re-estimating Y - D + Selected Variables
